In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk # natural language tool kit

In [2]:
initialFrame = pd.read_csv('train.tsv', delimiter = '\t');

In [3]:
initialFrame

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [4]:
# Cleans the initial frame
def cleanInitialFrame(df):
    cleanList = [] # list to grow
    currentSentence = 0 # tracks current sentence
    sentenceIDs = {0} 
    # Iterate row by row
    for index, row in df.iterrows():
        # If it's the first element, add to list
        if (row['SentenceId'] == currentSentence):
            continue
        else:
            cleanList.append([row['PhraseId'], row['SentenceId'], row['Phrase'], row['Sentiment']]);
            currentSentence = row['SentenceId']
    
    # Return a clean frame
    return pd.DataFrame(cleanList, columns = ['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'])

In [5]:
df = cleanInitialFrame(initialFrame)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,64,2,"This quiet , introspective and entertaining in...",4
2,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
3,117,4,A positively thrilling combination of ethnogra...,3
4,157,5,Aggressive self-glorification and a manipulati...,1


In [6]:
def lowerAllPhrases(df):
    phrases_list = list(df['Phrase'])

    for i in range(len(phrases_list)):
        phrases_list[i] = phrases_list[i].lower()
    count = 0;
    for index, row in df.iterrows():
        df.at[index,'Phrase'] = phrases_list[count]
        count += 1

    return df

In [7]:
lowerAllPhrases(df)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant 's work , i suspe...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1


In [8]:
# Remove non-ascii characters using str.replace()
def asciiClean(df):
    # iterate row by row
    for index, row in df.iterrows():
        old_str = row['Phrase']
        new_str = (old_str.encode('ascii','ignore')).decode()
        df.at[index, 'Phrase'] = new_str

In [9]:
asciiClean(df)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant 's work , i suspe...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1


In [10]:
df[0:10]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant 's work , i suspe...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1
5,167,6,a comedy-drama of nearly epic proportions root...,4
6,199,7,"narratively , trouble every day is a plodding ...",1
7,214,8,"the importance of being earnest , so thick wit...",3
8,248,9,but it does n't leave you with much .,1
9,260,10,you could hate it for the same reason .,1


In [11]:
import re
def removeSpaces(df):
    for index, row in df.iterrows():
        # df['Phrase'] = df['Phrase'].replace([row['Phrase']], re.sub(r'\s+\'', "'", row['Phrase']))
        df.at[index,'Phrase'] = re.sub(r'\s+\'', "'", row['Phrase'])

In [12]:
removeSpaces(df)

In [13]:
df[0:10]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant's work , i suspec...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1
5,167,6,a comedy-drama of nearly epic proportions root...,4
6,199,7,"narratively , trouble every day is a plodding ...",1
7,214,8,"the importance of being earnest , so thick wit...",3
8,248,9,but it does n't leave you with much .,1
9,260,10,you could hate it for the same reason .,1


In [14]:
import contractions
contractions.add('n\'t', 'not')
def expandContractions(df):
    for index, row in df.iterrows():
        phrase = []
        for i in row['Phrase'].split():
            phrase.append(contractions.fix(i))
        string_version = ' '.join(phrase)
        df.at[index, 'Phrase'] = string_version            

In [15]:
expandContractions(df)

In [16]:
import string
def removePunctuation(df):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    for index,row in df.iterrows():
        phrase = []
        for i in row['Phrase'].split():
            phrase.append(regex.sub('', i))
        string_version = ' '.join(phrase)
        df.at[index, 'Phrase'] = string_version

In [17]:
removePunctuation(df)

In [18]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andyv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andyv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
def removeStopWords(df):
    
    phrases_list = list(df['Phrase'])
    stop_words = set(stopwords.words('english'))

    for i in range(len(phrases_list)):
        word_tokens = word_tokenize(phrases_list[i])
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        phrases_list[i] = filtered_sentence
    
    for i in range(len(phrases_list)):
        phrases_list[i] = TreebankWordDetokenizer().detokenize(phrases_list[i])
    
    count = 0
    for index, row in df.iterrows():
        # df['Phrase'] = df['Phrase'].replace([row['Phrase']], phrases_list[count])
        df.at[index, 'Phrase'] = phrases_list[count]
        count += 1

    return df      

In [20]:
df.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,this quiet introspective and entertaining ind...,4
2,82,3,even fans of ismail merchants work i suspect ...,1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive selfglorification and a manipulativ...,1
5,167,6,a comedydrama of nearly epic proportions roote...,4
6,199,7,narratively trouble every day is a plodding m...,1
7,214,8,the importance of being earnest so thick with...,3
8,248,9,but it does not leave you with much,1
9,260,10,you could hate it for the same reason,1


In [21]:
df = removeStopWords(df)
df.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,series escapades demonstrating adage good goos...,1
1,64,2,quiet introspective entertaining independent w...,4
2,82,3,even fans ismail merchants work suspect would ...,1
3,117,4,positively thrilling combination ethnography i...,3
4,157,5,aggressive selfglorification manipulative whit...,1
5,167,6,comedydrama nearly epic proportions rooted sin...,4
6,199,7,narratively trouble every day plodding mess,1
7,214,8,importance earnest thick wit plays like readin...,3
8,248,9,leave much,1
9,260,10,could hate reason,1


In [22]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\andyv\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\andyv\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\andyv\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


In [23]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize     
def tag_words(df):
    allPhrases = list(df['Phrase'])
    for index,row in df.iterrows():
        tokenized = word_tokenize(allPhrases[index])
        tagged = nltk.pos_tag(tokenized)
        df.at[index, 'Phrase'] = tagged

def determine_root(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_phrases(df):
    lemmatizer = WordNetLemmatizer()
    allPhrases = list(df['Phrase'])
    for index,row in df.iterrows():
        final_sentence = []
        final_tag = list(map(lambda x: (x[0], determine_root(x[1])), allPhrases[index]))
        for word, tag in final_tag:
            if tag is None:
                final_sentence.append(word)
            else:
                final_sentence.append(lemmatizer.lemmatize(word, tag))
        final_sentence = ' '.join(final_sentence)
        df.at[index, 'Phrase'] = final_sentence             

In [24]:
tag_words(df)
lemmatize_phrases(df)

In [25]:
df.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,series escapades demonstrating adage good goos...,1
1,64,2,quiet introspective entertaining independent w...,4
2,82,3,even fan ismail merchant work suspect would ha...,1
3,117,4,positively thrill combination ethnography intr...,3
4,157,5,aggressive selfglorification manipulative whit...,1
5,167,6,comedydrama nearly epic proportion root sincer...,4
6,199,7,narratively trouble every day plod mess,1
7,214,8,importance earnest thick wit play like read ba...,3
8,248,9,leave much,1
9,260,10,could hate reason,1


In [26]:
# TO DO:
# * preprocessing
#     - replace all grammar with spaces
#     - lemmatize words
# * reduce to multi-dimensional vector
#     - We have options including: Bag of words
#     - BERT
#     - TF-IDF
# * classification
#     - try a bunch of classifiers
# * graphing
#     - graph the outputs of our classifiers
# * presentation
#     - create a presentation

In [27]:
class Selene(object):
    
    def __init__(self, dataFrame):
        
        self.df = dataFrame
        self.wordsDict = {}
        self.wordSentOcc = {}
        self.wordSentVal = {}
        
        # Tokenizes phrases and finds frequency & sentiment
        self.setUniqueWords()
        self.setFrequencies()
        self.findSentFrequency()
    
    # Finds unique words in the list of phrases and puts into list
    def setUniqueWords(self):
        
        phrases = list(self.df['Phrase'])
        
        for index, row in df.iterrows():
            sentence = row['Phrase']
            for w in word_tokenize(sentence): # word_tokens:
                if w.isalpha():
                    self.wordsDict[w] = [0,0,0,0,0,0]
                    
    def setFrequencies(self):
        
        for index, row in df.iterrows():
            sentence = row['Phrase']
            for word in sentence.split():
                if word in self.wordsDict:
                    self.wordsDict[word][0] += 1
                    
            
#         print("Here are all the unique words:")
        
#         for key, value in self.wordsDict.items():
#             print(key, ": ", value)
            
    
    # Finds frequencies of the sentiment per word
    def findSentFrequency(self):
                    
        for index, row in df.iterrows():
            sentiment = int(row['Sentiment'])
            for word in row['Phrase'].split():
                if word in self.wordsDict:
                    try:
                        self.wordsDict[word][sentiment + 1] += 1
                    except:
                        print(sentiment + 1)
                
        
        print("Words = [Frequency, 0 Freq, 1 Freq, 2 Freq, 3 Freq, 4 Freq, 5 Freq]")
        for key, value in self.wordsDict.items():
            print(key, ": ", value)
    # Finds probabilities for a word's sentiment
    def findProbabilities(self, key):
        
        prob0 = self.wordSentOcc[key][0 + 1] / self.wordsDict[key]
        prob1 = self.wordSentOcc[key][1 + 1] / self.wordsDict[key]
        prob2 = self.wordSentOcc[key][2 + 1] / self.wordsDict[key]
        prob3 = self.wordSentOcc[key][3 + 1] / self.wordsDict[key]
        prob4 = self.wordSentOcc[key][4 + 1] / self.wordsDict[key]
        return prob0, prob1, prob2, prob3, prob4
    # Finds the average sentiment of a word
    def findWordSentVal(self):
        
        for key, value in self.wordSentOcc.items():
            sentVal = (((value[0 + 1] * 0) + (value[1 + 1] * 1) + (value[2 + 1] * 2)
                    + (value[3 + 1] * 3) + (value[4 + 1] * 4)) / self.wordsDict[key])
            prob0, prob1, prob2, prob3, prob4 = self.findProbabilities(key)
            self.wordSentVal[key] = {'Avg Value': sentVal,'Probability of 0': prob0,
                                     'Probability of 1': prob1, 'Probability of 2': prob2, 
                                     'Probabilty of 3': prob3, 'Probability of 4': prob4}
            
        print("Here is the average value of the sentiment and probabilities of each word in the training data:")
        for key, value in self.wordSentVal.items():
            print(key, ": ", value)
            
    def getSentiment(self, testDataFrame, weightPercent = .1, minimumOccurances = 3):
        pass
                
                
        

In [28]:
test = Selene(df)
# test.findWordSentVal()

Words = [Frequency, 0 Freq, 1 Freq, 2 Freq, 3 Freq, 4 Freq, 5 Freq]
series :  [49, 10, 16, 7, 11, 5]
escapades :  [1, 0, 1, 0, 0, 0]
demonstrating :  [1, 0, 1, 0, 0, 0]
adage :  [3, 0, 1, 0, 1, 1]
good :  [302, 25, 80, 55, 96, 46]
goose :  [3, 1, 1, 1, 0, 0]
also :  [115, 12, 15, 21, 44, 23]
gander :  [1, 0, 1, 0, 0, 0]
occasionally :  [27, 0, 8, 8, 8, 3]
amuse :  [49, 2, 12, 7, 18, 10]
none :  [28, 6, 14, 5, 2, 1]
amount :  [25, 9, 7, 0, 8, 1]
much :  [263, 30, 97, 55, 53, 28]
story :  [378, 39, 94, 66, 124, 55]
quiet :  [24, 1, 3, 4, 10, 6]
introspective :  [3, 0, 0, 1, 1, 1]
entertaining :  [30, 1, 4, 3, 6, 16]
independent :  [4, 1, 1, 0, 1, 1]
worth :  [78, 3, 10, 17, 32, 16]
seek :  [15, 2, 5, 1, 2, 5]
even :  [258, 52, 72, 36, 69, 29]
fan :  [69, 2, 18, 13, 21, 15]
ismail :  [1, 0, 1, 0, 0, 0]
merchant :  [5, 0, 3, 1, 0, 1]
work :  [278, 25, 52, 41, 97, 63]
suspect :  [11, 1, 8, 0, 0, 2]
would :  [233, 40, 81, 49, 50, 13]
hard :  [92, 17, 34, 12, 20, 9]
time :  [300, 43, 81, 61, 

In [29]:
df.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,series escapades demonstrating adage good goos...,1
1,64,2,quiet introspective entertaining independent w...,4
2,82,3,even fan ismail merchant work suspect would ha...,1
3,117,4,positively thrill combination ethnography intr...,3
4,157,5,aggressive selfglorification manipulative whit...,1
5,167,6,comedydrama nearly epic proportion root sincer...,4
6,199,7,narratively trouble every day plod mess,1
7,214,8,importance earnest thick wit play like read ba...,3
8,248,9,leave much,1
9,260,10,could hate reason,1
